# NRM Configuration/Manifest guide

This notebook documents NRM's configuration and manifest format. The two next cells are for setup purposes.

In [1]:
%%capture
cd ..

In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

## Daemon configuration

`nrmd`'s configuration can be defined in the json/yaml/[Dhall](https://dhall-lang.org/) formats. Admissible values are defined in file [resources/configurationSchema.json](./resources/configurationSchema.json), and alternatively available as a Dhall type in [resources/types/Cfg.dhall](resources/types/Configuration.dhall). Schema files get large, so the next cells shows the Dhall Configuration type as a more readable alternative.

In [3]:
%%script dhall resolve
./resources/defaults/Cfg.dhall

{ argo_nodeos_config =
    "argo_nodeos_config"
, argo_perf_wrapper =
    "nrm-perfwrapper"
, controlCfg =
    None
    { learnCfg :
        < KnapsackConstraints :
            { _1 : { _1 : Double } }
        | LagrangeConstraints :
            { _1 : { _1 : Double } }
        >
    , minimumControlInterval :
        { fromuS : Double }
    , referenceMeasurementRoundInterval :
        Integer
    , speedThreshold :
        Double
    }
, downstreamCfg =
    { downstreamBindAddress = "ipc:///tmp/nrm-downstream-event" }
, dummy =
    True
, hwloc =
    "hwloc"
, hwmonCfg =
    { hwmonEnabled = True, hwmonPath = "/sys/class/hwmon" }
, libnrmPath =
    None Text
, logfile =
    "/tmp/nrm.log"
, nodeos =
    False
, perf =
    "perf"
, pmpi_lib =
    "pmpi_lib"
, raplCfg =
    None { raplFrequency : { fromHz : Double }, raplPath : Text }
, singularity =
    False
, slice_runtime =
    < Dummy | Nodeos | Singularity >.Dummy
, upstreamCfg =
    { pubPort = +2345, rpcPort = +3456, upstreamBi

<IPython.core.display.Javascript object>

Optional values are filled using defaults that can be found in [resources/defaults/Cfg.json](resources/defaults/Configuration.json) (also available in the Dhall format):

In [4]:
%%bash
cat ./resources/defaults/Cfg.json | jq

{
  "pmpi_lib": "pmpi_lib",
  "verbose": "error",
  "logfile": "/tmp/nrm.log",
  "singularity": false,
  "argo_nodeos_config": "argo_nodeos_config",
  "upstreamCfg": {
    "upstreamBindAddress": "*",
    "rpcPort": 3456,
    "pubPort": 2345
  },
  "perf": "perf",
  "argo_perf_wrapper": "nrm-perfwrapper",
  "downstreamCfg": {
    "downstreamBindAddress": "ipc:///tmp/nrm-downstream-event"
  },
  "nodeos": false,
  "hwloc": "hwloc",
  "dummy": true,
  "slice_runtime": "dummy",
  "hwmonCfg": {
    "hwmonPath": "/sys/class/hwmon",
    "hwmonEnabled": true
  }
}


<IPython.core.display.Javascript object>

## Manifest configuration

Example manifest files are in [resources/examples](../resources/examples) in JSON/YAML/Dhall format. For instance, the manifest file [resources/examples/perfwrap.json](../resources/examples/perfwrap.json) enables enables performance monitoring:

In [5]:
%%bash
cat resources/examples/perfwrap.json | jq

{
  "hwbind": false,
  "app": {
    "scheduler": {
      "fIFO": {}
    },
    "power": {
      "slowdown": 1,
      "profile": false,
      "policy": "noPowerPolicy"
    },
    "perfwrapper": {
      "perfwrapper": {
        "perfLimit": 100000,
        "perfFreq": 1
      }
    },
    "slice": {
      "cpus": 1,
      "mems": 1
    }
  },
  "name": "default"
}


<IPython.core.display.Javascript object>

Manifest options are documented in schema file [resources/manifestSchema.json](../resources/manifestSchema.json). The next cell shows the corresponding [Dhall](https://dhall-lang.org/) type.

In [6]:
%%script dhall resolve
./resources/types/Manifest.dhall

{ app :
    { instrumentation :
        Optional { ratelimit : { fromHz : Double } }
    , perfwrapper :
        < Perfwrapper :
            { _1 :
                { perfFreq :
                    { fromHz : Double }
                , perfLimit :
                    { fromOps : Integer }
                }
            }
        | PerfwrapperDisabled
        >
    , power :
        { policy :
            < Combined | DDCM | DVFS | NoPowerPolicy >
        , profile :
            Bool
        , slowdown :
            Integer
        }
    , scheduler :
        < FIFO | HPC | Other : { _1 : Integer } >
    , slice :
        { cpus : Integer, mems : Integer }
    }
, hwbind :
    Bool
, image :
    Optional
    { binds : Optional (List Text), imagetype : < Docker | Sif >, path : Text }
, name :
    Text
}


<IPython.core.display.Javascript object>

Under-specified manifests like the one in our `workloads` above (with missing optional fields from the schema) fill missing values with defaults, which are located in file [resources/defaults/Manifest.json](../../resources/examples/default.json):

In [7]:
%%bash
cat resources/defaults/Manifest.json | jq

{
  "hwbind": false,
  "app": {
    "scheduler": {
      "fIFO": {}
    },
    "power": {
      "slowdown": 1,
      "profile": false,
      "policy": "noPowerPolicy"
    },
    "perfwrapper": {
      "perfwrapperDisabled": {}
    },
    "slice": {
      "cpus": 1,
      "mems": 1
    }
  },
  "name": "default"
}


<IPython.core.display.Javascript object>

The `dhall` and `dhall-to-json` utilities are available as convenience in this environment should you need them. Dhall is useful as a configuration language in itself.

In [8]:
%%script dhall-to-json
let Manifest = ./resources/types/Manifest.dhall 
let appendName = \(m: Manifest) -> m // {name = m.name ++ "-appended" }
in appendName ./resources/defaults/Manifest.dhall

{"image":null,"hwbind":false,"app":{"scheduler":"FIFO","instrumentation":null,"power":{"slowdown":1,"profile":false,"policy":"NoPowerPolicy"},"perfwrapper":"PerfwrapperDisabled","slice":{"cpus":1,"mems":1}},"name":"default-appended"}


<IPython.core.display.Javascript object>

Remember that any json document is one little step away from being a Python dictionaryy:

In [10]:
import json

with open("resources/examples/perfwrap.json") as f:
    print(json.load(f))

{'hwbind': False, 'app': {'scheduler': {'fIFO': {}}, 'power': {'slowdown': 1, 'profile': False, 'policy': 'noPowerPolicy'}, 'perfwrapper': {'perfwrapper': {'perfLimit': 100000, 'perfFreq': 1}}, 'slice': {'cpus': 1, 'mems': 1}}, 'name': 'default'}


<IPython.core.display.Javascript object>